In [1]:
# Import libraries and packages for the project 
# import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup#pulling data out of HTML files
from time import sleep#slow down the execution to pass the security check
import csv
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('data/CUNY_students.csv')
df['Name']=df['LName']+df['FName']
df.head()

,LName,FName,Campus,Name
0,Cain,Gillian,Baruch,Cain Gillian
1,Jacques,Sabine,Baruch,Jacques Sabine
2,Lewis,Clyde,Baruch,Lewis Clyde
3,Alas,Ana,Baruch,Alas Ana
4,Oyo,Adashima,Baruch,Oyo Adashima


In [3]:
df['Name']=df['Name']+df['Campus']

names=[]
for i in list(df['Name']):
    cur=i.replace(u'\xa0', u' ')
    names.append(cur)
print(names)

[' Cain Gillian Baruch ', ' Jacques Sabine Baruch ', ' Lewis Clyde Baruch ', 'Alas Ana Baruch ', 'Oyo Adashima Baruch ', '\u202fKurton Beresford Baruch ', 'Greigg Walter BC ', 'Dastas Natalie BC ', 'Acco Giselle BMCC ', 'Bennett Sefton BMCC ', 'Blake  Andrew BMCC ', 'Nia Rene BMCC ', 'Paredes Jenny BMCC ', 'Onwumere Onyekwere Bronx CC ', 'Acosta Vadim  Bronx CC ', 'Allen Rasheen Bronx CC ', 'Carre  Jean-Marc Bronx CC ', 'Mighty Jason Bronx CC ', 'Bennett Nadine CCNY ', 'Enemuo Amarachukwu CCNY ', 'Many Winzy CCNY ', 'Baez  Ivan CCNY (Engr) ', 'Stanley  Nkem CCNY (Sci) ', 'Idowu Promise CSI ', 'Levano Kelly CSI ', 'Sidime      Francoise CSI ', 'Baidoo Juliet CSI ', 'Blaize Jonathan CSI ', 'Jmukhadze Irena CSI ', 'Levano Kelly CSI ', 'Dorcely Reginald Hostos ', 'Gbagba Adjiwanou Hostos ', 'Niet-Wire Clara Hostos ', 'Santiago Omar Hostos ', 'Miessein  Desire Hunter ', 'Orellana Alicia Hunter ', 'Padilla Angela Hunter ', 'Meyer Maria  Hunter ', 'Boyll Rachel JJAY ', 'Branker Melissa JJAY '

good Youtube videos:
1. [link]https://www.youtube.com/watch?v=zkfLAY2OrtI
2. [link]https://www.linkedin.com/learning/python-automation-and-testing/what-is-the-html-dom-structure?autoplay=true&resume=false

In [4]:
# Task 1: Login to Linkedin

credential=open('login_credential.txt')
line=credential.readlines()
username=line[0]
password=line[1]

# Task 1.1: Open Chrome and Access Linkedin login site
driver =webdriver.Chrome()
url = 'https://www.linkedin.com/login'
driver.get(url)
sleep(2)

# Task 1.2: Key in login credentials
email_field = driver.find_element('name','session_key')
email_field.send_keys(username)
sleep(2)

password_field = driver.find_element('id','password')
password_field.send_keys(password)
sleep(2)

# Task 1.2: Click the Login button

login_field = driver.find_element('xpath',"/html/body/div/main/div[2]/div[1]/form/div[3]/button")
login_field.click()
sleep(2)

In [5]:
# Task 2: Search for the profile we want to crawl

# Task 2.1: Locate the search bar element
# search_field = driver.find_element('xpath',"//button[@aria-label='Click to start a search'][@type='button']")
# search_field.click()

# Task 2.2: Input the search query to the search bar
#search_query = input("what do you want to search:")#using the list from Dr C
type_field=driver.find_element('xpath',"/html/body/div[5]/header/div/div/div/div[1]/input")
type_field.send_keys(names[0])

# Task 2.3: Search for CUNY student and click on People
type_field.send_keys(Keys.RETURN)
sleep(1)
driver.find_element('xpath','/html/body/div[5]/div[3]/div[2]/section/div/nav/div/ul/li[1]/button').click()
driver.find_element('xpath',"/html/body/div[5]/header/div/div/div/div[1]/input").clear()


In [6]:
# Task 3: Scrape the URLs of the profiles

# Task 3.1: Write a function to extract the URLs of one page
def GetURL():
    page_source = BeautifulSoup(driver.page_source)
    profiles = page_source.find_all('div',class_="entity-result__item") 


    all_profile_url=[]
    for profile in profiles:
        profile_url=profile.find('a',class_="app-aware-link")['href']
        if profile_url not in all_profile_url:
            all_profile_url.append(profile_url)
    return all_profile_url


   


# # Task 3.2: Navigate through the next pages, and extract the profile URLs of each page

# def GetURL_pages():
#     input_page = 1
#     URLs_all_page = []
#     for page in range(input_page):
#         URLs_one_page = GetURL()
#         driver.execute_script('window.scrollTo(0,document.body.scrollHeight)')#we need to scroll down to Next page button
#         sleep(1)
#         next_button = driver.find_element('xpath',"//button[@aria-label='Next'][@type='button']").click()
#         URLs_all_page = URLs_all_page + URLs_one_page
        
#         sleep(1)
#     return URLs_all_page

# URLs_all_page=GetURL_pages()
# print(URLs_all_page)

In [204]:
all_URLs=[]
all_URLs=np.concatenate((all_URLs,GetURL()),axis=None)

In [205]:
for i in range(1,len(names)):
    #step1: let us search for the person
    type_field=driver.find_element('xpath',"/html/body/div[5]/header/div/div/div/div[1]/input")
    type_field.send_keys(names[i])
    type_field.send_keys(Keys.RETURN)
    sleep(1)
    all_URLs=np.concatenate((all_URLs,GetURL()),axis=None)
    driver.find_element('xpath',"/html/body/div[5]/header/div/div/div/div[1]/input").clear()
    

In [212]:
URLs_all_page=[]

for i in all_URLs:
    if "https://www.linkedin.com/in/" in i:
        URLs_all_page.append(i)
print(URLs_all_page)


['https://www.linkedin.com/in/gillian-walker-03a870102?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAABocty4ByZYFg79ocWrzoXVxhqLOeQ7iN0o', 'https://www.linkedin.com/in/gillian-harrison-cain-mlis-cae-42b3033?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAACUKqMBznFlREFjGSS5bSFsF9-VLSviouc', 'https://www.linkedin.com/in/gillian-cain-b112281a8?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADBxdj0BhyV_FB7Xpc6JNd_KYKY1Dlyv2Sc', 'https://www.linkedin.com/in/gillian-cain-8862bb11?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAAJbsiYBbKujpmacKKBhnDmExL3D9bLdkHg', 'https://www.linkedin.com/in/gillian-cain-62795b220?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADerjr8BL8VO5Wz3llECNJfksOdU3CHC9G4', 'https://www.linkedin.com/in/gillian-cain-684761223?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADgstIQBDyY4Bbi2zRUhh0KiIg5NjRYFgtg', 'https://www.linkedin.com/in/gillian-cain-56221627?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAAWMhJoB73p0okeJC9Z5uPrHlfEv8rsTQrw', 'https://www.linkedin.co

In [214]:
# Task 4: Scrape the data of 1 Linkedin profile, and write the data to a .CSV file
with open('output.csv', 'w',  newline = '') as file_output:
    headers = ['Name', 'Job Title', 'Location','Degrees','Schools','Work Places','Work Titles','URL',]
    writer = csv.DictWriter(file_output, delimiter=',', lineterminator='\n',fieldnames=headers)
    writer.writeheader()
    
    for linkedin_URL in URLs_all_page:
        driver.get(linkedin_URL)
#         print(linkedin_URL)
        sleep(1)
        page_source = BeautifulSoup(driver.page_source, "html.parser")
        info_div = page_source.find('div',{'class':'mt2 relative'})

        name = info_div.find('h1', class_='text-heading-xlarge inline t-24 v-align-middle break-words').get_text().strip() #Remove unnecessary characters 
        #print('--- Profile name is: ', name)
        location = info_div.find('span', class_='text-body-small inline t-black--light break-words').get_text().strip() #Remove unnecessary characters 
        #print('--- Profile location is: ', location)
        title = info_div.find('div', class_='text-body-medium break-words').get_text().strip()
        #print('--- Profile title is: ', title)
        
        working_titles=[]
        working_places=[]
        if page_source.find('div',{'id':'experience'}):
            work_exp=page_source.find('div',{'id':'experience'}).find_parent().find('ul',{'class':'pvs-list ph5 display-flex flex-row flex-wrap'})
            for i in work_exp.find_all('span',class_='mr1 t-bold'):
                working_titles.append(i.get_text())
            for i in work_exp.find_all('span',class_='t-14 t-normal'):
                working_places.append(i.get_text())
        
        
        degrees=[]
        schools=[]
        if page_source.find('div',{'id':'education'}):
            ed_exp=page_source.find('div',{'id':'education'}).find_parent().find('ul',{'class':'pvs-list ph5 display-flex flex-row flex-wrap'})
            for i in ed_exp.find_all('span',class_='t-14 t-normal'):
                degrees.append(i.get_text())
            for j in ed_exp.find_all('span',class_='mr1 hoverable-link-text t-bold'):
                schools.append(j.get_text())
        
        writer.writerow({headers[0]:name, headers[1]:location, headers[2]:title,
                         headers[3]:degrees, headers[4]:schools,
                         headers[5]:working_places,headers[6]:working_titles, headers[3]:linkedin_URL})
        print('\n')
    
        



AttributeError: 'NoneType' object has no attribute 'find'

In [195]:
driver.get('https://www.linkedin.com/in/gillian-walker-03a870102?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAABocty4ByZYFg79ocWrzoXVxhqLOeQ7iN0o')
page_source = BeautifulSoup(driver.page_source, "html.parser")
work_exp=page_source.find('div',{'id':'experience'}).find_parent().find('ul',{'class':'pvs-list ph5 display-flex flex-row flex-wrap'})
working_titles=[]
for i in work_exp.find_all('span',class_='mr1 t-bold'):
    working_titles.append(i.get_text())
print(working_titles)

working_places=[]
for i in work_exp.find_all('span',class_='t-14 t-normal'):
    working_places.append(i.get_text())
print(working_places)

ed_exp=page_source.find('div',{'id':'education'}).find_parent().find('ul',{'class':'pvs-list ph5 display-flex flex-row flex-wrap'})
degrees=[]
schools=[]
for i in ed_exp.find_all('span',class_='t-14 t-normal'):
    degrees.append(i.get_text())
for j in ed_exp.find_all('span',class_='mr1 hoverable-link-text t-bold'):
    schools.append(j.get_text())
print(degrees)
print(schools)

['\nProject Engineer/Resident EngineerProject Engineer/Resident Engineer\n', '\nCivil EngineerCivil Engineer\n', '\nGraduate Research AssistantGraduate Research Assistant\n', '\nResearch AssistantResearch Assistant\n']
['\nHVEA EngineersHVEA Engineers\n', '\nNaik Consulting Group, PCNaik Consulting Group, PC\n', '\nCity University of New York Institute for Transportation SystemsCity University of New York Institute for Transportation Systems\n', '\nCE Department, Environmental Engineering Lab, The City College of New YorkCE Department, Environmental Engineering Lab, The City College of New York\n']
['\nMaster of Engineering (MEng), Civil Engineering - TransportationMaster of Engineering (MEng), Civil Engineering - Transportation\n', '\nBachelor of Engineering (BE), Civil EngineeringBachelor of Engineering (BE), Civil Engineering\n']
['\nThe City College of New YorkThe City College of New York\n', '\nThe City College of New YorkThe City College of New York\n']


In [209]:
print(all_URLs)

['https://www.linkedin.com/in/gillian-walker-03a870102?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAABocty4ByZYFg79ocWrzoXVxhqLOeQ7iN0o'
 'https://www.linkedin.com/in/gillian-harrison-cain-mlis-cae-42b3033?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAACUKqMBznFlREFjGSS5bSFsF9-VLSviouc'
 'https://www.linkedin.com/in/gillian-cain-b112281a8?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADBxdj0BhyV_FB7Xpc6JNd_KYKY1Dlyv2Sc'
 'https://www.linkedin.com/in/gillian-cain-8862bb11?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAAJbsiYBbKujpmacKKBhnDmExL3D9bLdkHg'
 'https://www.linkedin.com/in/gillian-cain-62795b220?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADerjr8BL8VO5Wz3llECNJfksOdU3CHC9G4'
 'https://www.linkedin.com/in/gillian-cain-684761223?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADgstIQBDyY4Bbi2zRUhh0KiIg5NjRYFgtg'
 'https://www.linkedin.com/in/gillian-cain-56221627?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAAWMhJoB73p0okeJC9Z5uPrHlfEv8rsTQrw'
 'https://www.linkedin.co